In [ ]:
import os
import pandas as pd

In [ ]:
# >Iterate DAISEEsplitted/0 and save the name of each video in a dataframe
df = pd.DataFrame(columns=['ClipID', 'label'])
for i in range(0, 4):
    path = '../DAiSEE/DataSet/Splitted/' + str(i) + '/'
    for filename in os.listdir(path):
        # if end with .avi or .mp4, append to dataframe
        if filename.endswith('.avi') or filename.endswith('.mp4'):
            new_row = pd.DataFrame({'ClipID': [filename], 'label': [i]})
            df = pd.concat([df, new_row], ignore_index=True)

In [ ]:
def split_labels(dataframe,label, frac=0.7, limit=1500):
    # Obtain the number of videos
    n = dataframe[dataframe["label"] == label].shape[0]
    # obtain the limit of videos randomly
    df = dataframe[dataframe['label'] == label].sample(n=limit, random_state=123) if n > 1500 else dataframe[dataframe['label'] == label].copy()
    n = df.shape[0]
    # Divide the dataframe into train for label 0 and label 1
    train = df.sample(frac=frac, random_state=123)
    # get the rest of the data for validation
    val= df.drop(train.index).sample(frac=0.5, random_state=123)
    # get the test data
    test = df.drop(train.index).drop(val.index)

    # print the number of videos for each label
    print(f"Number of videos for label '{label}': " + str(n))
    print(f"Number of videos for train label '{label}': " + str(train.shape[0]))
    print(f"Number of videos for validation label '{label}': " + str(val.shape[0]))
    print(f"Number of videos for test label '{label}': " + str(test.shape[0]))
    print("\n------------------------------------------------------------\n")
    return n,train,val,test

In [ ]:
n0, train_0, val_0,test_0 = split_labels(df,0)
n1, train_1, val_1,test_1 = split_labels(df,1)
n2, train_2, val_2,test_2 = split_labels(df,2)
n3, train_3, val_3,test_3 = split_labels(df,3)

In [ ]:
Class0Path = "../DAiSEE/DataSet/Aug/0/"
Class1Path = "../DAiSEE/DataSet/Aug/1/"
Class2Path = "../DAiSEE/DataSet/Aug/2/"
Class3Path = "../DAiSEE/DataSet/Aug/3/"
# Create paths
if not os.path.exists(Class0Path):
    os.makedirs(Class0Path)
if not os.path.exists(Class1Path):
    os.makedirs(Class1Path)
if not os.path.exists(Class2Path):
    os.makedirs(Class2Path)
if not os.path.exists(Class3Path):
    os.makedirs(Class3Path)

In [ ]:
import os
def video_augmentation(classPath, train, augmentations):
    videos = train["ClipID"].tolist()
    pathSplitted = "../DAiSEE/DataSet/Splitted/" + classPath[-2]
    # create folder for training
    if not os.path.exists(os.path.join(classPath, "train")):
        os.makedirs(os.path.join(classPath, "train")) 
    for v in videos:
        video_path = os.path.join(pathSplitted, v)

        if not os.path.exists(video_path):
            print(f"Warning: Video {video_path} not found, skipping.")
            continue

        for i, aug in enumerate(augmentations):
            aug_name = aug["name"]
            save_name = f"train/aug{i:04d}-{v}"
            saved_video = os.path.join(classPath, save_name)

            if os.path.exists(saved_video):
                print(f"Video {saved_video} already exists, skipping.")
                continue

            try:
                aug["function"](video_path, saved_video)
                print(f"Saved {aug_name} augmented video: {saved_video}")
            except Exception as e:
                print(f"Error applying {aug_name} to {v}: {str(e)}")
                
def save_videos(classPath, df, folder):
    videos = df["ClipID"].tolist()
    pathSplitted = "../DAiSEE/DataSet/Splitted/" + classPath[-2]
    if not os.path.exists(os.path.join(classPath, folder)):
        os.makedirs(os.path.join(classPath, folder)) 
    for v in videos:
        video_path = os.path.join(pathSplitted, v)

        if not os.path.exists(video_path):
            print(f"Warning: Video {video_path} not found, skipping.")
            continue

        save_name = f"{folder}/{v}"
        saved_video = os.path.join(classPath, save_name)

        if os.path.exists(saved_video):
            print(f"Video {saved_video} already exists, skipping.")
            continue

        try:
            os.system(f"cp {video_path} {saved_video}")
            print(f"Saved video: {saved_video}")
        except Exception as e:
            print(f"Error saving video {v}: {str(e)}")

# Augmentations

In [ ]:
from augly.video import functional as vf

def combine_augmentations(aug1, aug2):
    """Combina dos aumentaciones"""
    def combined(src, dest):
        temp_file = src + ".temp.mp4"
        aug1(src, temp_file)
        aug2(temp_file, dest)
        if os.path.exists(temp_file):
            os.remove(temp_file)
    return combined

## LAbel 0

In [ ]:
import shutil
augmentations = [
    {"name": "original", "function": lambda src, dest: shutil.copy(src, dest)},
    {
        "name": "-brightness",
        "function": lambda src, dest: vf.brightness(
            video_path=src, output_path=dest, level=0.5
        ),
    },
    {
        "name": "brightness",
        "function": lambda src, dest: vf.brightness(
            video_path=src, output_path=dest, level=-0.25
        ),
    },
    {
        "name": "blur",
        "function": lambda src, dest: vf.blur(
            video_path=src, output_path=dest, sigma=5.25
        ),
    },
    {
        "name": "rotate",
        "function": lambda src, dest: vf.rotate(
            video_path=src, output_path=dest, degrees=35
        ),
    },
    # NEWS
    {
        "name": "contrast",
        "function": lambda src, dest: vf.contrast(
            video_path=src, output_path=dest, level=5.5
        ),
    },
    {
        "name": "-contrast",
        "function": lambda src, dest: vf.contrast(
            video_path=src, output_path=dest, level=-5.5
        ),
    },
    {
        "name": "grayscale",
        "function": lambda src, dest: vf.grayscale(video_path=src, output_path=dest),
    },
    {
        "name": "noise",
        "function": lambda src, dest: vf.add_noise(video_path=src, output_path=dest, level=50),
    },
    {
        "name": "pixelization",
        "function": lambda src, dest: vf.pixelization(
            video_path=src, output_path=dest, ratio=0.15
        ),
    },
    {
        "name": "color_jitter",
        "function": lambda src, dest: vf.color_jitter(
            video_path=src,
            output_path=dest,
            brightness_factor=0.1,
            contrast_factor=0.1,
            saturation_factor=2,
        ),
    },
    # Añadir combinaciones
    {
        "name": "grayscale_blur",
        "function": combine_augmentations(
            lambda s, d: vf.grayscale(video_path=s, output_path=d),
            lambda s, d: vf.blur(video_path=s, output_path=d, sigma=5),
        ),
    },
    {
        "name": "rotate_brightness",
        "function": combine_augmentations(
            lambda s, d: vf.rotate(video_path=s, output_path=d, degrees=25),
            lambda s, d: vf.brightness(video_path=s, output_path=d, level=0.5),
        ),
    },
    {
        "name": "rotate__45_pixelation",
        "function": combine_augmentations(
            lambda s, d: vf.rotate(video_path=s, output_path=d, degrees=-45),
            lambda s, d: vf.pixelization(video_path=s, output_path=d, ratio=0.15),
        ),
    },
    {
        "name": "rotate_45_blur",
        "function": combine_augmentations(
            lambda s, d: vf.rotate(video_path=s, output_path=d, degrees=45),
            lambda s, d: vf.blur(video_path=s, output_path=d, sigma=5.5),
        ),
    },
    # more combinations
    {
        "name": "brightness_contrast",
        "function": combine_augmentations(
            lambda s, d: vf.brightness(video_path=s, output_path=d, level=0.5),
            lambda s, d: vf.contrast(video_path=s, output_path=d, level=1.5),
        ),
    },
    {
        "name": "grayscale_noise",
        "function": combine_augmentations(
            lambda s, d: vf.grayscale(video_path=s, output_path=d),
            lambda s, d: vf.add_noise(video_path=s, output_path=d),
        ),
    },
    {
        "name": "crop_pixelization",
        "function": combine_augmentations(
            lambda s, d: vf.crop(
                video_path=s,
                output_path=d,
                left=0.1,
                top=0.1,
                right=0.9,
                bottom=0.9,
            ),
            lambda s, d: vf.pixelization(video_path=s, output_path=d, ratio=0.15),
        ),
    },
    {
        "name": "color_jitter_blur",
        "function": combine_augmentations(
            lambda s, d: vf.color_jitter(
                video_path=s,
                output_path=d,
                brightness_factor=0.1,
                contrast_factor=0.2,
                saturation_factor=1.2,
            ),
            lambda s, d: vf.blur(video_path=s, output_path=d, sigma=5.5),
        ),
    },
    {
        "name": "color_jitter_rotate",
        "function": combine_augmentations(
            lambda s, d: vf.color_jitter(
                video_path=s,
                output_path=d,
                brightness_factor=0.1,
                contrast_factor=0.2,
                saturation_factor=1.2,
            ),
            lambda s, d: vf.rotate(video_path=s, output_path=d, degrees=25),
        ),
    },
    {
        "name": "color_jitter_crop",
        "function": combine_augmentations(
            lambda s, d: vf.color_jitter(
                video_path=s,
                output_path=d,
                brightness_factor=0.1,
                contrast_factor=0.1,
                saturation_factor=1.2,
            ),
            lambda s, d: vf.crop(
                video_path=s,
                output_path=d,
                left=0.1,
                top=0.1,
                right=0.9,
                bottom=0.9,
            ),
        ),
    },
    {
        "name": "color_jitter_pixelization",
        "function": combine_augmentations(
            lambda s, d: vf.color_jitter(
                video_path=s,
                output_path=d,
                brightness_factor=0.1,
                contrast_factor=0.1,
                saturation_factor=1.2,
            ),
            lambda s, d: vf.pixelization(video_path=s, output_path=d, ratio=0.15),
        ),
    },
    {
        "name": "color_jitter_noise",
        "function": combine_augmentations(
            lambda s, d: vf.color_jitter(
                video_path=s,
                output_path=d,
                contrast_factor=0.5,
                saturation_factor=1.5,
            ),
            lambda s, d: vf.add_noise(video_path=s, output_path=d),
        ),
    },
]

In [ ]:
# #! Ejecutar para class 0
video_augmentation(Class0Path, train_0, augmentations)
save_videos(Class0Path, test_0, "test")
save_videos(Class0Path, val_0, "validation")

In [ ]:
train_1.shape

## Label 1

In [ ]:
import shutil
augmentations = [
    {"name": "original", "function": lambda src, dest: shutil.copy(src, dest)},
    {
        "name": "-brightness",
        "function": lambda src, dest: vf.brightness(
            video_path=src, output_path=dest, level=0.5
        ),
    },
    # NEWS
    {
        "name": "-contrast",
        "function": lambda src, dest: vf.contrast(
            video_path=src, output_path=dest, level=-5.5
        ),
    },
    # Añadir combinaciones
    {
        "name": "rotate_brightness",
        "function": combine_augmentations(
            lambda s, d: vf.rotate(video_path=s, output_path=d, degrees=25),
            lambda s, d: vf.brightness(video_path=s, output_path=d, level=0.5),
        ),
    },
    {
        "name": "rotate__45_pixelation",
        "function": combine_augmentations(
            lambda s, d: vf.rotate(video_path=s, output_path=d, degrees=-45),
            lambda s, d: vf.pixelization(video_path=s, output_path=d, ratio=0.15),
        ),
    },
    {
        "name": "rotate_45_blur",
        "function": combine_augmentations(
            lambda s, d: vf.rotate(video_path=s, output_path=d, degrees=45),
            lambda s, d: vf.blur(video_path=s, output_path=d, sigma=5.5),
        ),
    },
]

In [ ]:
# #! Ejecutar para class 1
video_augmentation(Class1Path, train_1, augmentations)
save_videos(Class1Path, test_1, "test")
save_videos(Class1Path, val_1, "validation")

## Label 2
Dont need augmentation.

In [ ]:
save_videos(Class2Path, train_2, "train")
save_videos(Class2Path, test_2, "test")
save_videos(Class2Path, val_2, "validation")

## Label 3
Dont need augmentation

In [ ]:
save_videos(Class3Path, train_3, "train")
save_videos(Class3Path, test_3, "test")
save_videos(Class3Path, val_3, "validation")